# Análise Multissetorial - Piauí

Este notebook demonstra a integração e análise de dados de múltiplos setores governamentais do estado do Piauí.

## Objetivos

1. Carregar dados de diferentes fontes (Saúde, Educação, Economia, Assistência Social)
2. Explorar correlações entre indicadores setoriais
3. Identificar municípios com melhores e piores indicadores
4. Criar visualizações comparativas

## Fontes de Dados

- **Saúde**: DATASUS (SIM, SINASC, SIH)
- **Educação**: INEP (Censo Escolar, IDEB)
- **Economia**: IBGE (PIB, CEMPRE)
- **Assistência Social**: MDS (CadÚnico, Bolsa Família)

In [ ]:
# Imports
import sys
from pathlib import Path

# Adicionar diretório raiz ao path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Bibliotecas carregadas com sucesso!")

## 1. Geração de Dados Sintéticos

Primeiro, vamos gerar os dados sintéticos para demonstração.

In [ ]:
from src.extractors import SyntheticDataGenerator

# Gerar dados
generator = SyntheticDataGenerator()
datasets = generator.generate_all()

print(f"Datasets gerados: {list(datasets.keys())}")
for name, df in datasets.items():
    print(f"  - {name}: {len(df)} registros, {len(df.columns)} colunas")

## 2. Exploração dos Dados de Saúde

In [ ]:
# Dados de mortalidade
df_mortalidade = datasets['mortalidade']
print("\n=== MORTALIDADE ===")
print(f"Período: {df_mortalidade['ano'].min()} - {df_mortalidade['ano'].max()}")
print(f"\nColunas: {df_mortalidade.columns.tolist()}")
df_mortalidade.head()

In [ ]:
# Evolução da taxa de mortalidade infantil
fig = px.line(
    df_mortalidade.groupby('ano')['taxa_mortalidade_infantil'].mean().reset_index(),
    x='ano',
    y='taxa_mortalidade_infantil',
    title='Evolução da Taxa de Mortalidade Infantil - Piauí',
    labels={'ano': 'Ano', 'taxa_mortalidade_infantil': 'Taxa por 1.000 nascidos vivos'}
)
fig.update_traces(mode='lines+markers')
fig.show()

In [ ]:
# Dados de nascimentos
df_nascimentos = datasets['nascimentos']
print("\n=== NASCIMENTOS ===")
df_nascimentos.head()

In [ ]:
# Distribuição por tipo de parto
parto_stats = df_nascimentos.groupby('ano').agg({
    'partos_normais': 'sum',
    'partos_cesareos': 'sum'
}).reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(name='Normal', x=parto_stats['ano'], y=parto_stats['partos_normais']))
fig.add_trace(go.Bar(name='Cesáreo', x=parto_stats['ano'], y=parto_stats['partos_cesareos']))
fig.update_layout(
    title='Tipos de Parto por Ano - Piauí',
    barmode='group',
    xaxis_title='Ano',
    yaxis_title='Número de Partos'
)
fig.show()

## 3. Exploração dos Dados de Educação

In [ ]:
# Dados de escolas
df_escolas = datasets['escolas']
print("\n=== ESCOLAS ===")
print(f"Total de escolas: {len(df_escolas)}")
df_escolas.head()

In [ ]:
# Escolas por tipo e rede
fig = px.sunburst(
    df_escolas,
    path=['tipo_escola', 'rede'],
    values='total_alunos',
    title='Distribuição de Alunos por Tipo de Escola e Rede - Piauí'
)
fig.show()

In [ ]:
# Dados do IDEB
df_ideb = datasets['ideb']
print("\n=== IDEB ===")
df_ideb.head()

In [ ]:
# Evolução do IDEB por nível de ensino
ideb_medio = df_ideb.groupby(['ano', 'nivel_ensino'])['ideb'].mean().reset_index()

fig = px.line(
    ideb_medio,
    x='ano',
    y='ideb',
    color='nivel_ensino',
    title='Evolução do IDEB por Nível de Ensino - Piauí',
    labels={'ano': 'Ano', 'ideb': 'IDEB', 'nivel_ensino': 'Nível de Ensino'}
)
fig.update_traces(mode='lines+markers')
fig.show()

## 4. Exploração dos Dados Econômicos

In [ ]:
# Dados de PIB
df_pib = datasets['pib_municipal']
print("\n=== PIB MUNICIPAL ===")
df_pib.head()

In [ ]:
# Top 10 municípios por PIB
top_pib = df_pib[df_pib['ano'] == df_pib['ano'].max()].nlargest(10, 'pib_total_mil_reais')

fig = px.bar(
    top_pib,
    x='municipio_nome',
    y='pib_total_mil_reais',
    title=f'Top 10 Municípios por PIB - Piauí ({top_pib["ano"].iloc[0]})',
    labels={'municipio_nome': 'Município', 'pib_total_mil_reais': 'PIB (R$ mil)'},
    color='pib_per_capita'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# Composição do PIB (Teresina)
teresina_pib = df_pib[(df_pib['municipio_nome'] == 'Teresina') & (df_pib['ano'] == df_pib['ano'].max())].iloc[0]

setores = ['agropecuaria', 'industria', 'servicos', 'impostos']
valores = [teresina_pib[f'va_{s}_mil_reais'] for s in setores]

fig = px.pie(
    values=valores,
    names=['Agropecuária', 'Indústria', 'Serviços', 'Impostos'],
    title=f'Composição do PIB de Teresina ({teresina_pib["ano"]})'
)
fig.show()

## 5. Exploração dos Dados de Assistência Social

In [ ]:
# Dados do CadÚnico
df_cadunico = datasets['cadunico']
print("\n=== CADASTRO ÚNICO ===")
df_cadunico.head()

In [ ]:
# Evolução de famílias no CadÚnico
cadunico_ano = df_cadunico.groupby('ano').agg({
    'familias_cadastradas': 'sum',
    'familias_extrema_pobreza': 'sum',
    'familias_pobreza': 'sum'
}).reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=cadunico_ano['ano'], 
    y=cadunico_ano['familias_cadastradas'],
    name='Total Cadastradas',
    fill='tozeroy'
))
fig.add_trace(go.Scatter(
    x=cadunico_ano['ano'], 
    y=cadunico_ano['familias_extrema_pobreza'],
    name='Extrema Pobreza'
))
fig.add_trace(go.Scatter(
    x=cadunico_ano['ano'], 
    y=cadunico_ano['familias_pobreza'],
    name='Pobreza'
))
fig.update_layout(
    title='Evolução de Famílias no Cadastro Único - Piauí',
    xaxis_title='Ano',
    yaxis_title='Número de Famílias'
)
fig.show()

## 6. Análise Correlacionada Multissetorial

Agora vamos integrar os dados dos diferentes setores para análises cruzadas.

In [ ]:
# Criar dataset integrado por município e ano
ano_ref = 2023

# PIB
pib_2023 = df_pib[df_pib['ano'] == ano_ref][['municipio_id', 'municipio_nome', 'pib_per_capita', 'populacao']]

# Mortalidade
mort_2023 = df_mortalidade[df_mortalidade['ano'] == ano_ref][['municipio_id', 'taxa_mortalidade_infantil']]

# IDEB (média por município)
ideb_2023 = df_ideb[df_ideb['ano'] == ano_ref].groupby('municipio_id')['ideb'].mean().reset_index()
ideb_2023.columns = ['municipio_id', 'ideb_medio']

# CadÚnico
cad_2023 = df_cadunico[df_cadunico['ano'] == ano_ref][['municipio_id', 'familias_cadastradas', 'familias_extrema_pobreza']]
cad_2023['pct_extrema_pobreza'] = cad_2023['familias_extrema_pobreza'] / cad_2023['familias_cadastradas'] * 100

# Merge
df_integrado = pib_2023.merge(mort_2023, on='municipio_id', how='left')
df_integrado = df_integrado.merge(ideb_2023, on='municipio_id', how='left')
df_integrado = df_integrado.merge(cad_2023[['municipio_id', 'pct_extrema_pobreza']], on='municipio_id', how='left')

print(f"Dataset integrado: {len(df_integrado)} municípios")
df_integrado.head(10)

In [ ]:
# Matriz de correlação
cols_analise = ['pib_per_capita', 'taxa_mortalidade_infantil', 'ideb_medio', 'pct_extrema_pobreza']
corr_matrix = df_integrado[cols_analise].corr()

fig = px.imshow(
    corr_matrix,
    labels=dict(color="Correlação"),
    x=['PIB per Capita', 'Mort. Infantil', 'IDEB', '% Ext. Pobreza'],
    y=['PIB per Capita', 'Mort. Infantil', 'IDEB', '% Ext. Pobreza'],
    color_continuous_scale='RdBu_r',
    title='Matriz de Correlação - Indicadores Multissetoriais (Piauí 2023)'
)
fig.update_layout(width=600, height=500)
fig.show()

In [ ]:
# Scatter: PIB per Capita vs Taxa de Mortalidade Infantil
fig = px.scatter(
    df_integrado,
    x='pib_per_capita',
    y='taxa_mortalidade_infantil',
    size='populacao',
    hover_name='municipio_nome',
    trendline='ols',
    title='PIB per Capita vs Taxa de Mortalidade Infantil',
    labels={
        'pib_per_capita': 'PIB per Capita (R$)',
        'taxa_mortalidade_infantil': 'Taxa Mort. Infantil (por 1.000)'
    }
)
fig.show()

In [ ]:
# Scatter: IDEB vs Extrema Pobreza
fig = px.scatter(
    df_integrado,
    x='pct_extrema_pobreza',
    y='ideb_medio',
    size='populacao',
    hover_name='municipio_nome',
    trendline='ols',
    title='% Extrema Pobreza vs IDEB Médio',
    labels={
        'pct_extrema_pobreza': '% Famílias em Extrema Pobreza',
        'ideb_medio': 'IDEB Médio'
    }
)
fig.show()

## 7. Ranking Multissetorial de Municípios

In [ ]:
# Criar índice composto (normalizado)
from sklearn.preprocessing import MinMaxScaler

df_rank = df_integrado.copy()

# Normalizar indicadores (0-1)
scaler = MinMaxScaler()

# Para mortalidade e pobreza, inverter (menor é melhor)
df_rank['pib_norm'] = scaler.fit_transform(df_rank[['pib_per_capita']])
df_rank['mort_norm'] = 1 - scaler.fit_transform(df_rank[['taxa_mortalidade_infantil']])
df_rank['ideb_norm'] = scaler.fit_transform(df_rank[['ideb_medio']])
df_rank['pobreza_norm'] = 1 - scaler.fit_transform(df_rank[['pct_extrema_pobreza']])

# Índice composto (média simples)
df_rank['indice_desenvolvimento'] = (
    df_rank['pib_norm'] + 
    df_rank['mort_norm'] + 
    df_rank['ideb_norm'] + 
    df_rank['pobreza_norm']
) / 4

# Ranking
df_rank = df_rank.sort_values('indice_desenvolvimento', ascending=False)
df_rank['ranking'] = range(1, len(df_rank) + 1)

print("\n=== TOP 10 MUNICÍPIOS ===")
df_rank[['ranking', 'municipio_nome', 'indice_desenvolvimento', 'pib_per_capita', 'ideb_medio', 'taxa_mortalidade_infantil']].head(10)

In [ ]:
# Visualização: Top 10 vs Bottom 10
top10 = df_rank.head(10)
bottom10 = df_rank.tail(10)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 10 Municípios', 'Bottom 10 Municípios'))

fig.add_trace(
    go.Bar(x=top10['municipio_nome'], y=top10['indice_desenvolvimento'], marker_color='green'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=bottom10['municipio_nome'], y=bottom10['indice_desenvolvimento'], marker_color='red'),
    row=1, col=2
)

fig.update_layout(
    title='Índice de Desenvolvimento Municipal Composto - Piauí 2023',
    showlegend=False,
    height=500
)
fig.update_xaxes(tickangle=-45)
fig.show()

## 8. Conclusões

### Principais Insights

1. **Correlação PIB x Mortalidade**: Municípios com maior PIB per capita tendem a ter menores taxas de mortalidade infantil.

2. **Educação x Pobreza**: Existe correlação negativa entre o IDEB e a porcentagem de famílias em extrema pobreza.

3. **Desigualdade Regional**: A capital Teresina concentra grande parte do PIB estadual, evidenciando a necessidade de políticas de descentralização.

4. **Oportunidades**: Municípios no bottom 10 do ranking de desenvolvimento podem ser priorizados para investimentos públicos.

### Próximos Passos

- Integração com dados geográficos para análise espacial
- Séries temporais para previsão de indicadores
- Dashboard interativo para gestores públicos